In [1]:
!pip install ultralytics opencv-contrib-python segment-anything cvlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 7.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.1/874.1 kB 7.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 MB 14.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.6 MB/s eta 0:00:0000:0100:01
  Created wheel for cvlib: filename=cvlib-0.2.7-py3-none-any.whl size=10046370 sha256=8e44fcfc3157566bcef4485522c6d48aff723908367d931aebb59684b802fccd
  Stored in directory: /Users/ronitkhurana/Library/Caches/pip/wheels/83/07/aa/65f82dedaf511cdd3ff55778ddb8867161334ac786e52651e0
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-a

In [2]:
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox

In [3]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!wget -O /content/drive/MyDrive/yolov3.cfg https://github.com/pjreddie/darknet/raw/master/cfg/yolov3.cfg
!wget -O /content/drive/MyDrive/yolov3.weights https://pjreddie.com/media/files/yolov3.weights

In [ ]:
import cv2
import numpy as np

weights = "/content/drive/MyDrive/yolov3.weights"
config = "/content/drive/MyDrive/yolov3.cfg"
net = cv2.dnn.readNet(weights, config)

img = cv2.imread("/content/puppy-5124947_640.jpg")

height, width, _ = img.shape
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
outs = net.forward(output_layers)

class_ids = []
confidences = []
boxes = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.3:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.3,0.4)

with open("/content/drive/MyDrive/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

def draw_bbox(img, boxes, class_ids, confidences, indices):
    for i in indices:
        if isinstance(i, np.ndarray):
            i = i[0]
        box = boxes[i]
        (x, y, w, h) = box
        label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    return img

output_image = draw_bbox(img, boxes, class_ids, confidences, indices)

cv2.imwrite("output.jpg", output_image)

In [ ]:
print(h)

In [ ]:
import torch
from segment_anything import SamPredictor, sam_model_registry
import matplotlib.pyplot as plt
import numpy as np

model_type = "vit_h"
sam_checkpoint = "/content/drive/MyDrive/sam_vit_h_4b8939.pth"
device = "cuda" if torch.cuda.is_available() else "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)



In [ ]:
image_path = "/content/puppy-5124947_640.jpg"
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

predictor.set_image(image_rgb)


In [ ]:

padding = 50
input_box = np.array([x-padding,y-padding,x+w+padding,y+h+padding])

input_box = torch.tensor(input_box).unsqueeze(0).numpy()

masks, _, _ = predictor.predict(box=input_box)

plt.figure(figsize=(10, 10))
plt.imshow(image_rgb)

for mask in masks:
    plt.imshow(mask, alpha=0.2)

plt.axis('off')
plt.show()

In [ ]:
image_path = "/content/puppy-5124947_640.jpg"
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Assuming masks[0] is a boolean mask for the subject
mask = masks[0]

# Create an RGBA image
height, width, _ = image_rgb.shape
subject_image_rgba = np.zeros((height, width, 4), dtype=np.uint8)

# Set RGB channels where the mask is true
subject_image_rgba[..., :3] = np.where(mask[..., np.newaxis], image_rgb, 0)

# Set the alpha channel: fully opaque where the mask is true, fully transparent otherwise
subject_image_rgba[..., 3] = np.where(mask, 255, 0)

# Display the image
plt.figure(figsize=(10, 10))
plt.imshow(subject_image_rgba)
plt.axis('off')
plt.show()

# Save the image with transparency

cv2.imwrite("subject.png", cv2.cvtColor(subject_image_rgba, cv2.COLOR_RGBA2BGRA))

In [ ]:
# mask = masks[0]  # Assuming we take the first mask if multiple masks exist


# # Convert mask to 3-channel if your image is in RGB (shape: [H, W, 3])
# mask_3_channel = np.stack([mask]*3, axis=-1)  # Convert to 3-channel for RGB

# # Set the object area (where mask is True) to black
# image_with_black = np.where(mask_3_channel, [0, 0, 0], image_rgb)

# # Display the modified image
# plt.figure(figsize=(10, 10))
# plt.imshow(image_with_black)
# plt.axis('off')
# plt.show()

# # Convert the result image to uint8 (ensure values are in the range [0, 255])
# image_with_black_uint8 = image_with_black.astype(np.uint8)

# # Save the modified image
# cv2.imwrite("modified_image.jpg", cv2.cvtColor(image_with_black_uint8, cv2.COLOR_RGB2BGR))

In [ ]:
!pip install diffusers transformers accelerate
!pip install torch torchvision
!pip install opencv-python-headless

In [ ]:
import torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
import numpy as np
import cv2

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float32
).to(device)


In [ ]:

def cv2_to_pil(img):
    return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

def pil_to_cv2(pil_img):
    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

In [ ]:

# image_rgb = cv2.imread("modified_image.jpg")
# image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB)

# mask = masks[0]

# binary_mask = (mask * 255).astype(np.uint8)

# image_pil = Image.fromarray(image_rgb)
# mask_pil = Image.fromarray(binary_mask)

# plt.figure(figsize=(10, 10))
# plt.imshow(binary_mask, cmap="gray")
# plt.axis('off')
# plt.show()

In [ ]:
rect_mask = np.zeros((image_rgb.shape[0], image_rgb.shape[1]), dtype=np.uint8)
rect_mask[y-padding:y+h+padding, x-padding:x+w+padding] = 255

mask_pil = Image.fromarray(rect_mask)

plt.figure(figsize=(10, 10))
plt.imshow(mask_pil, cmap="gray")
plt.axis('off')
plt.show()


In [ ]:
from matplotlib import pyplot as plt
prompt = "green grass, natural landscape, blending seamlessly with the rest of the park"
negative_prompt = "no animals, no unnatural elements, no objects"

inpainted_image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=image_pil,
    mask_image=mask_pil,
    num_inference_steps=50,
    guidance_scale=7.5
).images[0]

plt.figure(figsize=(10, 10))
plt.imshow(inpainted_image)
plt.axis('off')
plt.show()

inpainted_image.save("final_inpainted_image.jpg")